In [1]:
#### ###
%config InlineBackend.figure_format = 'retina'
%pylab inline
import sys
sys.path.insert(1, '../codes/')
from geeCodes import *
from myUtils import *
from sklearn.preprocessing import MinMaxScaler

Populating the interactive namespace from numpy and matplotlib


In [2]:
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')
dfc = dfc.to_crs("EPSG:4326")
target_vars = ['hot_days', 'hot_nights']

In [6]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3959 entries, 0 to 3958
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   hot_days    3959 non-null   int64   
 1   lat         3959 non-null   float64 
 2   lon         3959 non-null   float64 
 3   hot_nights  3959 non-null   int64   
 4   NDVI        3959 non-null   float64 
 5   NDBI        3959 non-null   float64 
 6   Albedo_BSA  3959 non-null   float64 
 7   Albedo_WSA  3959 non-null   float64 
 8   dist        3959 non-null   float64 
 9   pop         3959 non-null   float64 
 10  city        3959 non-null   object  
 11  geometry    3959 non-null   geometry
dtypes: float64(8), geometry(1), int64(2), object(1)
memory usage: 371.3+ KB


In [10]:
idx = np.arange(2,3)
gdf_all = pd.DataFrame()
for ii in idx:
    gdf = gpd.read_file('../data_revision/GEE_dataframes/gdf_%d.shp'%ii)
    print(gdf['NDVI'].min())
    gdf =  gdf.set_crs('epsg:4326')
    gdf = gdf.to_crs("ESRI:54009")
    
    gdf['dist'] = -gdf['dist']
    gdf = gdf[gdf['dist']>0]
    gdf['dist_n'] = 1/(gdf['dist']**2)
    gdf['dist_n'] = norm01(gdf, 'dist_n')
    gdf = gdf[gdf['NDVI']>=-1]
    gdf = gdf[gdf['NDBI']>=-1].reset_index(drop=True)
    gdf['hot_days']=gdf['hot_days']/11
    gdf['hot_nights']=gdf['hot_nights']/11
    gdf_all = gdf_all.append(gdf, ignore_index=True)

-999.0


In [ ]:
w_train = libpysal.weights.KNN.from_dataframe(gdf_all, k=8)
w_train.transform = 'r'

In [ ]:
y_day_train = gdf_all[target_vars[0]].values.reshape(-1, 1)
predictor_vars = ['NDVI', 'dist_n']
y_day_train = gdf_all[target_vars[0]].values.reshape(-1, 1)
x_train = gdf_all[predictor_vars].values
slm_day = GM_Lag(y_day_train, x_train, w=w_train, 
                     name_y=target_vars[0], name_x=predictor_vars, spat_diag=True)

In [ ]:
print(slm_day.summary)

In [ ]:
##################### Training Validation ###########################################


######### Load datasets for cross validation
dfTV = pd.read_pickle('../data_revision/cities/cv/df_train_val.pkl')
dfTest = pd.read_pickle('../data_revision/cities/cv/df_test.pkl')
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')

In [ ]:
def valSet(gdfF, cc, XVars, Yvars):
    gdf_val = gdfF[gdfF['city'].isin(cc)].reset_index(drop=True)    
    y_day_val = gdf_val[Yvars[0]].values.reshape(-1, 1)
    y_night_val= gdf_val[Yvars[1]].values.reshape(-1, 1)
    x_val = gdf_val[XVars].values
    
    ### Scale #################################################
    #scaler = StandardScaler()
    #scaler.fit(y_day_val)
    #y_day_val = scaler.transform(y_day_val)
    
    #scaler0 = StandardScaler()
    #scaler0.fit(y_night_val)
    #y_night_val = scaler0.transform(y_night_val)

    #scaler1 = StandardScaler()
    #scaler1.fit(x_val)
    #x_val = scaler1.transform(x_val)
    
    
    w_val = libpysal.weights.KNN.from_dataframe(gdf_val, k=8)
    w_val.transform = 'r'
    w_day_val   = libpysal.weights.lag_spatial(w_val, y_day_val)
    w_night_val = libpysal.weights.lag_spatial(w_val, y_night_val)
    return w_day_val, w_night_val, x_val, y_day_val, y_night_val


def trainSet(gdfF, cc, XVars, Yvars):
    gdf_train = gdfF[~gdfF['city'].isin(cc)].reset_index(drop=True)
    w_train = libpysal.weights.KNN.from_dataframe(gdf_train, k=8)
    w_train.transform = 'r'
    
    x_train = gdf_train[XVars].values
    y_day_train = gdf_train[Yvars[0]].values.reshape(-1, 1)
    y_night_train = gdf_train[Yvars[1]].values.reshape(-1, 1)
        
    ### Scale #################################################
    #scaler = StandardScaler()
    #scaler.fit(y_day_train)
    #y_day_train = scaler.transform(y_day_train)
    
    #scaler0 = StandardScaler()
    #scaler0.fit(y_night_train)
    #y_night_train = scaler0.transform(y_night_train)

    #scaler1 = StandardScaler()
    #scaler1.fit(x_train)
    #x_train = scaler1.transform(x_train)

    return w_train, x_train, y_day_train, y_night_train

def run_model_validation_s(x_, w, model):
    val = sm.add_constant(np.hstack((x_, np.array(w).reshape(-1, 1))))
    return np.sum(val * model.betas.T, axis=1).reshape((-1, 1))

def run_model_validation_o(x_, model):
    val = sm.add_constant(x_)
    return np.sum(val* model.betas.T, axis=1).reshape((-1, 1))


from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
count = 0
cols = ['Phase', 'R2_train_slm', 'R2_val_slm', 'R2_train_ols', 
        'R2_val_ols', 'MAE_val_slm', 'MAE_val_ols', 'SLM', 'OLS']
df_res_day   = pd.DataFrame(columns=cols)
df_res_night = pd.DataFrame(columns=cols)
for phase in range(0, 5):
    cities_test = list(dfTest[dfTest['phase']==phase]['UC_NM_MN'])
    gdfF = gdf_all[~gdf_all['city'].isin(cities_test)].reset_index(drop=True)
    for subphase in range(0, 5):
        count+=1
        subset = list(dfTV[(dfTV['phase']==phase) & ((dfTV['subphase']==subphase))]['UC_NM_MN'])
        
        
        ###################################################################################################
        ####################### Training set ##############################################################
        w_train, x_train, y_day_train, y_night_train = trainSet(gdfF, subset, predictor_vars, target_vars)


        ####################### Day #######################################################################
        slm_day = GM_Lag(y_day_train, x_train, w=w_train, 
                         name_y=target_vars[0], name_x=predictor_vars, spat_diag=True)
        ols_day = OLS(y_day_train, x_train, w=w_train, 
                      name_y=target_vars[0], name_x=predictor_vars, spat_diag=True)

        ####################### Night #####################################################################
        slm_night = GM_Lag(y_night_train, x_train, w=w_train, 
                         name_y=target_vars[1], name_x=predictor_vars, spat_diag=True)
        ols_night = OLS(y_night_train, x_train, w=w_train, 
                      name_y=target_vars[1], name_x=predictor_vars, spat_diag=True)


        #####################################################################################################
        ###################### Validation set ###############################################################
        w_day_val, w_night_val, x_val, y_day_val, y_night_val =  valSet(gdfF, subset, 
                                                                        predictor_vars, target_vars)



        ####################### Day ##########################################################################
        ySLM_day = run_model_validation_s(x_val, w_day_val, slm_day)
        yOls_day = run_model_validation_o(x_val, ols_day)

        ####################### Night ##########################################################################
        ySLM_night = run_model_validation_s(x_val, w_night_val, slm_night)
        yOls_night = run_model_validation_o(x_val, ols_night)



        data = [phase, slm_day.pr2, r2_score(ySLM_day, y_day_val), ols_day.r2, 
                r2_score(yOls_day, y_day_val), mean_absolute_error(ySLM_day, y_day_val), 
                mean_absolute_error(yOls_day, y_day_val),
                slm_day.betas, ols_day.betas]
        df_res_day = df_res_day.append(pd.DataFrame(columns=cols,data=[data]), ignore_index=True)

        data = [phase, slm_night.pr2, r2_score(ySLM_night, y_night_val), ols_night.r2, 
                r2_score(yOls_night, y_night_val), mean_absolute_error(ySLM_night, y_night_val), 
                mean_absolute_error(yOls_night, y_night_val),
                slm_night.betas, ols_night.betas]
        df_res_night = df_res_night.append(pd.DataFrame(columns=cols,data=[data]), ignore_index=True)
        end = time.time()
        print(r2_score(ySLM_night, y_night_val))
        print(r2_score(ySLM_day, y_day_val))
        print(slm_night.betas.T)
        print(slm_day.betas.T)   

In [ ]:
df_res_day.to_pickle('../data_revision/results/df_res_day_new.pkl')
df_res_night.to_pickle('../data_revision/results/df_res_night_new.pkl')